In [90]:
import numpy as np
import pandas as pd

In [91]:
n_car = 12 * 60 // 15  ## 12小时共需检修 48 量机车

n_workshop_A = 3    # 工序a车间数
n_workshop_B = 8    # 工序b车间数
n_workshop_C = 5    # 工序c车间数

time_cost_A = 4     ## 4*15 分钟
time_cost_B = 8     ## 8*15 分钟
time_cost_C = 6     ## 6*15 分钟

## 本问中所有工序时间以 15分钟为一单位
M = 1000  ## 任意足够大的时间数，作遍历用

input_times = dict() ## 创建一个字典，表示在车间足够多的情况下，动车的进入时间
for i in range(n_car):
    input_times[i] = 1 ## 1表示改时间点有动车进入

## 工序A

In [92]:
is_finish_A = np.zeros( n_car , dtype = int) # 记录所有动车的工艺完成状态，1为已完成，0为未完成
is_work_A = np.zeros(shape = ( M , n_workshop_A), dtype = int)  ## 记录车间 [a1 ; a2 ; a3] 三个车间每单位时间的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序的时间
output_time_record = []  ## 记录每个动车完成工序的时间
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间编号

n_arrival_car = 0  ## 初始化检修完的车辆数

for time in range(M):
    num_work_A = is_work_A[time].sum()  ## 当前时间车间的运作数量
    num_finish_A = is_finish_A.sum()      ## 已经发出的机车数量
    if num_work_A == 0 and num_finish_A == n_car: ## 工艺完成
        break
    else:
        if num_work_A < n_workshop_A and time in input_times:## 有空闲车间 且 有新车进
            free_id = np.where(is_work_A[time] == 0)[0][0]
            is_work_A[time: time + time_cost_A, free_id] = 1
            is_finish_A[n_arrival_car] = 1
            n_arrival_car += 1
            #print( time ,free_id )
            output_time_record.append(time + time_cost_A)
            input_time_record.append(time)
            workshop_ind_record.append(free_id)
        
        if num_work_A == n_workshop_A and time in input_times :## 无空闲车间 且 有新车进，则推迟新车进来  
            flag_input = False
            for k in range(10000):
                if time + k not in input_times:
                    input_times[time + k] = 1
                    break
            

df_A = pd.DataFrame({'进站时间': input_time_record,
                     '出站时间': output_time_record,
                     '检修车间编号':workshop_ind_record})

df_A = df_A[[ '进站时间','出站时间','检修车间编号']]

df_A['检测耗时(分钟)'] = df_A['出站时间'] - df_A['进站时间']

df_A['检修车间编号(展示)'] = df_A['检修车间编号'] + 1
#df_A.to_csv('render.csv', encoding = 'gbk' , index = None)

## 工序B

In [93]:
input_times = dict() ## 完成上一道工序后的出站时间为下一道工序的紧张时间
for output_time in df_A['出站时间']:
    input_times[output_time] = 1 ## 1表示改时间点有动车进入

is_finish_B = np.zeros( n_car , dtype = int) # 记录所有动车的工艺完成状态，1为已完成，0为未完成
is_work_B = np.zeros(shape = ( M , n_workshop_B), dtype = int)  ## 记录车间每单位时间的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序的时间
output_time_record = []  ## 记录每个动车完成工序的时间
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间编号

n_arrival_car = 0  ## 初始化检修完的车辆数

for time in range(M):
    num_work_B = is_work_B[time].sum()  ## 当前时间车间的运作数量
    num_finish_B = is_finish_B.sum()      ## 已经发出的机车数量
    if num_work_B == 0 and num_finish_B == n_car: ## 工艺完成
        break
    else:
        if num_work_B < n_workshop_B and time in input_times:## 有空闲车间 且 有新车进
            free_id = np.where(is_work_B[time] == 0)[0][0]
            is_work_B[time: time + time_cost_B, free_id] = 1
            is_finish_B[n_arrival_car] = 1
            n_arrival_car += 1
            #print( time ,free_id )
            output_time_record.append(time + time_cost_B)
            input_time_record.append(time)
            workshop_ind_record.append(free_id)
        
        if num_work_B == n_workshop_B and time in input_times :## 无空闲车间 且 有新车进，则推迟新车进来  
            flag_input = False
            for k in range(10000):
                if time + k not in input_times:
                    input_times[time + k] = 1
                    break
            

df_B = pd.DataFrame({'进站时间': input_time_record,
                     '出站时间': output_time_record,
                     '检修车间编号':workshop_ind_record})

df_B = df_B[[ '进站时间','出站时间','检修车间编号']]

df_B['检测耗时(分钟)'] = df_B['出站时间'] - df_B['进站时间']

df_B['检修车间编号(展示)'] = df_B['检修车间编号'] + 1
#df_B.to_csv('render.csv', encoding = 'gbk' , index = None)

## 工序C

In [98]:
input_times = dict() ## 完成上一道工序后的出站时间为下一道工序的紧张时间
for output_time in df_B['出站时间']:
    input_times[output_time] = 1 ## 1表示改时间点有动车进入

is_finish_C = np.zeros( n_car , dtype = int) # 记录所有动车的工艺完成状态，1为已完成，0为未完成
is_work_C = np.zeros(shape = ( M , n_workshop_C), dtype = int)  ## 记录车间每单位时间的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序的时间
output_time_record = []  ## 记录每个动车完成工序的时间
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间编号

n_arrival_car = 0  ## 初始化检修完的车辆数

for time in range(M):
    num_work_C = is_work_C[time].sum()  ## 当前时间车间的运作数量
    num_finish_C = is_finish_C.sum()      ## 已经发出的机车数量
    if num_work_C == 0 and num_finish_C == n_car: ## 工艺完成
        break
    else:
        if num_work_C < n_workshop_C and time in input_times:## 有空闲车间 且 有新车进
            free_id = np.where(is_work_C[time] == 0)[0][0]
            is_work_C[time: time + time_cost_C, free_id] = 1
            is_finish_C[n_arrival_car] = 1
            n_arrival_car += 1
            #print( time ,free_id )
            output_time_record.append(time + time_cost_C)
            input_time_record.append(time)
            workshop_ind_record.append(free_id)
        
        if num_work_C == n_workshop_C and time in input_times :## 无空闲车间 且 有新车进，则推迟新车进来  
            flag_input = False
            for k in range(10000):
                if time + k not in input_times:
                    input_times[time + k] = 1
                    break
            

df_C = pd.DataFrame({'进站时间': input_time_record,
                     '出站时间': output_time_record,
                     '检修车间编号':workshop_ind_record})

df_C = df_C[[ '进站时间','出站时间','检修车间编号']]

df_C['检测耗时(分钟)'] = df_C['出站时间'] - df_C['进站时间']

df_C['检修车间编号(展示)'] = df_C['检修车间编号'] + 1
#df_C.to_csv('render.csv', encoding = 'gbk' , index = None)

### 计算总耗时

In [105]:
(df_C['出站时间'].max() - df_A['进站时间'].min() )* 15 / 60 ## 小时

20.0